# SLIDE (1) Матричные производные

На вход подаются вектор $\boldsymbol{x}$ длины $n$, $\boldsymbol{c}$ длины $m$ и матрица $A$ размера $\{n\times m\}$

Пусть $y = \boldsymbol{x}^TA\boldsymbol{c}$

Найдите $\frac{dy}{d\boldsymbol{x}}$, $\frac{dy}{dA}$ и $\frac{dy}{d\boldsymbol{c}}$

# TASK

In [2]:
def matrix_deriv(x: np.array, A: np.array, c: np.array):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [35]:
######################################################
x = np.array([ [1], [2], [3],  [4]])
c = np.array([[25], [5],[-9]])
A = np.arange(12).reshape(4,3)

y1, y2, y3 = matrix_deriv(x, A, c)

assert_array_equal(y1, np.array([[-13],
                                [ 50],
                                [113],
                                [176]]))

assert_array_equal(y2, np.array([[ 25,   5,  -9],
                                [ 50,  10, -18],
                                [ 75,  15, -27],
                                [100,  20, -36]]))
                   
assert_array_equal(y3, np.array([[60, 70, 80]]))
######################################################


# SLIDE (1) Честная регрессия

В случае линейной регрессии по функционалу $MSE$ задачу оптимизации можно записать следующим образом:

$$ \frac{1}{n}\sum_{k=1}^N (\boldsymbol{x}_k^T\boldsymbol{w} - y_k) ^ 2 \to \min_{\boldsymbol{w}}$$

Мы уже знаем, что решение будет выглядеть следующим образом:

$$\boldsymbol{w} = \begin{cases} X^{-1}\boldsymbol{y}, & n = m\\
(X^TX)^{-1}X^T\boldsymbol{y}, & n > m\\
X^{T}(XX^{T})^{-1}\boldsymbol{y}, & n < m
\end{cases}$$

где $n$ - количество объектов, а $m$ - количество признаков + 1 (дополнительно 1 вес без признака).

Решите задачу регрессии честно, используя формулу выше. 

Чтобы вы не запутались мы сразу написали добавление единичного столбца к матрице $X$.


### Sample 1
#### Input:
```python
X_train = np.array([[1]])
y_train = np.array([2])
X_test = np.array([[0.], [2.], [3.]])
```
#### Output:
```python
model.predict(X_test) == np.array([1., 3., 4.])
model.w_ == np.array([[1.], [1.]])
model.coef_ == np.array([1., 1.])
```

# TASK

In [2]:
class TrueLinReg():
    def __init__(self):
        self.w_ = None  # столбец
        self.coef_ = None # строка
        
    def fit(self, X: np.array, y: np.array) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1) #добавляем вес без признака
        
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        self.coef_ = self.w_.reshape(-1)
        return self
        
    def predict(self, X) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        return (X @ self.w_).reshape(-1)

In [37]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = TrueLinReg().fit(X_reg, y_reg)

assert_array_almost_equal(model.predict(np.array([[0], [3], [4]])), np.array([0, 3, 4]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[1.], [0.]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([1., 0.]), decimal=2)
######################################################
X_reg = np.array([[1], [2], [3]])
y_reg = np.array([1, -2, 1])

model = TrueLinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[0],[4]])), np.array([0., 0.]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[0.], [0.]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([0., 0.]), decimal=2)
######################################################
X_reg = np.array([[1]])
y_reg = np.array([2])

model = TrueLinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[0.], [2.], [3.]])), np.array([1., 3., 4.]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[1.], [1.]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([1., 1.]), decimal=2)
######################################################
X_reg, y_reg = make_regression(n_samples=10, n_features=9, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model_my = TrueLinReg().fit(X_reg, y_reg)

coef_real = np.hstack([model.coef_, model.intercept_])

assert_array_almost_equal(model_my.coef_, coef_real, decimal=3)
######################################################
X_reg, y_reg = make_regression(n_samples=200, n_features=10, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model_my = TrueLinReg().fit(X_reg, y_reg)

coef_real = np.hstack([model.coef_, model.intercept_])

assert_array_almost_equal(model_my.coef_, coef_real, decimal=3)
######################################################
X_reg, y_reg = make_regression(n_samples=10, n_features=15, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model_my = TrueLinReg().fit(X_reg, y_reg)
coef_real = np.hstack([model.coef_, model.intercept_])

assert sklearn.metrics.mean_absolute_error(model_my.coef_, coef_real) < 20

# SLIDE (1) Честная регуляризация

Добавьте регуляризацию с коэффициентом $\lambda$ и решите задачу регрессии (для любого соотношения $m$ и $n$ используйте формулу)
$$\boldsymbol{w} = (X^TX + \lambda E)^{-1}X^T\boldsymbol{y}$$
где $E$ - единичная матрица.

Не забудьте сами добавить **справа** единичный столбец к матрице $X$ аналогично предыдущей задаче.
### Sample 1
#### Input:
```python
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])
lamb = 1

X_test = np.array([[0], [4]])
```
#### Output:
```python
model.predict(X_test) == np.array([0.33, 3])
model.w_ == np.array([[0.67], [0.33]])
model.coef_ == np.array([0.67, 0.33])
```

# TASK

In [5]:
class TrueReg():
    def __init__(self, lamb):
        self.lamb = lamb
        
    def fit(self, X, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
        
    def predict(self, X):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass

In [39]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = TrueReg(1).fit(X_reg, y_reg)

assert_array_almost_equal(model.predict(np.array([[0], [4]])), np.array([0.33, 3]), decimal=2)

assert_array_almost_equal(model.w_, np.array([[0.67], [0.33]]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([0.67, 0.33]), decimal=2)
######################################################
X_reg, y_reg = make_regression(n_samples=100, n_features=99, n_targets=1)

model = Ridge(1).fit(X_reg, y_reg)
model_my = TrueReg(1).fit(X_reg, y_reg)

coef_real = np.hstack([model.coef_, model.intercept_])

assert_array_almost_equal(model_my.coef_, coef_real, decimal=0)
######################################################
X_reg, y_reg = make_regression(n_samples=2000, n_features=10, n_targets=1)

model = Ridge(1).fit(X_reg, y_reg)
model_my = TrueReg(1).fit(X_reg, y_reg)

coef_real = np.hstack([model.coef_, model.intercept_])

assert_array_almost_equal(model_my.coef_, coef_real, decimal=0)
######################################################
X_reg, y_reg = make_regression(n_samples=100, n_features=150, n_targets=1)

model = Ridge(1).fit(X_reg, y_reg)
model_my = TrueReg(1).fit(X_reg, y_reg)
coef_real = np.hstack([model.coef_, model.intercept_])

assert sklearn.metrics.mean_absolute_error(model_my.coef_, coef_real) < 20

# SLIDE (1) Градиент

На вход подаются обучающая выборка $(X,\boldsymbol{y})$ и как-то определенные веса $w$. Верните градиент функции $MSE$ для изменения весов в алгоритме градиентного спуска.
$$\nabla_{\boldsymbol{w}} L = \frac{2}{n} X^{T}(X\boldsymbol{w} - \boldsymbol{y})$$

Здесь не нужно добавлять единицы сбоку к матрице, считаем, что они уже есть.

### Sample 1
#### Input:
```python
w = np.array([[0.2], [0.2]]) #столбец!
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[2],[3]]) #столбец!
```
#### Output:
```python
array([[-8.4], 
       [-8.4]]) #возвращаем столбец!
```

# TASK

In [39]:
def gradient_step(w: np.array, X: np.array, y: np.array) -> np.array:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [9]:
w = np.array([[0.2], 
              [0.2]])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[2],[3]])
assert_array_almost_equal( gradient_step(w, X, y), np.array([[-5.6],[-5.6]]))
######################################################
w = np.array([[-1.90059439], [-0.39772511], [-0.41752917], [ 1.52698708],
              [ 1.93384661], [ 0.55038402], [-0.43914043], [-0.53450491],
              [-0.20436079], [ 1.47930748], [-0.43431298], [-0.67715663],
              [ 0.6140469 ], [ 0.33724021], [-0.47389417], [-0.27931867],
              [-0.19815547], [-1.10817221], [-0.2783202 ], [-0.26673129]])

y = np.array([[-1.35285676], [-1.90639143], [-1.1872863 ], [-0.47531246],
              [-0.5658066 ], [-2.01777789], [-0.55111576], [-1.17508504],
              [-0.31591632], [-1.10876769]])

X = np.array([[-1.79418438, -0.28247459,  0.63485396,  0.18973693, -1.13052678,
                -0.70693165,  0.02485169, -1.32269407, -1.19972889, -0.90835517,
                1.70735351, -0.31278511,  0.26195237,  1.15186451, -1.06028399,
                0.35805288,  0.1097917 , -1.64881113, -0.26764338,  0.52419335],
              [-0.39820007, -1.07706627,  0.03495314, -0.36069043, -0.38655026,
                0.99804715,  0.62738777,  1.53889357,  0.47893302, -1.17044503,
                -1.14344307,  0.99980883, -0.89283077, -0.30342722, -1.64897791,
                -1.13511464, -0.82071543,  1.8879236 , -1.5129215 ,  0.12872281],
              [-0.26032624, -0.59442306, -0.20607555,  1.0785192 , -0.72817743,
                1.60260617, -2.34932775,  0.55693193,  0.25573628,  1.53147877,
                1.53834672,  0.1878024 ,  1.18042765,  0.54514643,  0.01803304,
                1.51429761, -0.17469223,  0.81986536, -0.46578708,  0.42436921],
              [ 0.10641183, -0.18608143,  2.69988291,  0.30114113, -0.56763953,
                -0.89837188,  1.19577105, -0.86920194, -2.68921283, -0.58902866,
                -0.14408206, -1.84817003, -0.9124478 ,  0.55288917,  1.14731704,
                -0.4114787 , -0.53848335,  0.63225441,  0.61095749, -0.16311904],
              [ 0.42409436,  1.00368214, -1.47558928,  0.01692064,  0.95318017,
                -0.04567367, -1.62167744, -0.20345068, -0.1389324 , -0.4342349 ,
                0.41877207, -1.37897995, -0.31973986, -1.6112183 , -0.18485078,
                -1.16219592, -1.08054026, -0.49210254, -1.52373135,  2.34733614],
              [ 0.19339171, -1.25994886, -1.3263278 , -1.56807527, -1.16484596,
                1.51848938, -0.40378262,  0.28889234,  0.10067917,  1.89112864,
                -1.87355328,  0.19510271,  1.13826496,  0.41762094,  0.61912712,
                -1.06733394, -0.22801313, -1.49211563, -0.97819834, -0.08353847],
              [ 0.5225887 ,  0.03288573, -0.1318777 ,  0.75014314, -1.36193554,
                -0.71094454,  2.09153476,  1.50715026,  0.03406876, -0.71503137,
                1.22186058,  0.9721076 , -0.33217577,  0.60094908, -0.19959933,
                -0.79264798, -0.80242992, -1.46592485,  0.00955109, -0.31271658],
              [ 1.07854603,  1.63387859, -0.68689403, -1.49026312, -0.05352863,
                0.74923652, -0.61221446,  0.25499529, -0.87862293,  2.30763454,
                -0.45654804, -0.49537167, -1.14185717,  0.09358794,  1.71154247,
                0.51549091, -0.26392429,  1.16472574, -0.53397584, -0.79062073],
              [-0.56480156, -1.83123029,  0.99887682, -0.60648359,  0.37281722,
                0.19015213,  0.08486153,  1.54633375, -0.22768608,  1.4061507 ,
                -0.07928461, -1.03523357,  0.44072784, -0.98970029, -0.10722007,
                1.54788238, -0.89526339, -1.14606415, -0.65972113,  0.96906237],
              [ 1.36608419,  0.09914266, -0.02822534,  0.97303833,  2.14361812,
                0.18005144,  0.94756695,  1.97038803, -1.00026315,  0.47603231,
                1.14925524, -1.16030033,  1.42723652,  0.00458275,  0.62078139,
                -2.47565446, -0.16698078,  0.25653049,  0.8664139 , -0.37928587]])


######################################################
g = np.array([-1.0993842672390932, -3.0877233608129395, -2.13575686356165, -0.21916813495587567, 
              1.6843662270764397, 3.3322365301217958, -5.049140901669979, 1.5001184420835205, 
              -0.4856689267508395, 4.84907077228488, 1.8736602701312628, -2.8537960600078045, 
              5.534114292930919, -0.6315258126682273, -0.10153263395507897, 0.10430238830850164, 
              -0.6327318737851857, -3.3518163054239145, -1.9647292933263618, 3.1983699740883607])

assert np.allclose(gradient_step(w, X, y).reshape(20), g)
######################################################

# SLIDE (1) Стохастический градиент

Реализуйте стохаистический градиентный спуск: пересчитайте $\nabla_{\boldsymbol{w}}L$ только для одного случайно выбранного элемента из выборки $X$.

$$L_{k} = (\boldsymbol{x}_k^T \boldsymbol{w} - y_k) ^ 2 \to \min_{\boldsymbol{w}}$$

$$\nabla_{\boldsymbol{w}}L_{k} = \boldsymbol{x}_k (\boldsymbol{x}_k^T \boldsymbol{w}  - y_k)$$
где $\boldsymbol{x}_i$ - вектор объекта выборки, выбранный случайно.  


Здесь не нужно добавлять единицы сбоку к матрице $X$, считаем, что они уже есть.

### Sample 1
#### Input:
```python
w = np.array([[1], [1], [1]]) #столбец!
X = np.array([[1, 1, 1], [0, 0, 0]])
y = np.array([[1], [0]]) #столбец!
```
#### Output:
```python
array([[0.],
       [0.],
       [0.]]) #столбец!
или
array([[2.],
       [2.],
       [2.]]) #столбец!
```

# TASK

In [36]:
import numpy as np

def stochastic_step(w: np.array, X: np.array, y: np.array) -> np.array:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [58]:
w = np.array([[1], [1], [1]])
y = np.array([[1], [0]])
X = np.array([[1, 1, 1], [0, 0, 0]])

z , k = 0 , 0
for i in range(100):
    g = stochastic_step(w, X, y)
    if g[0] == 2:
        z += 1
    else:
        k += 1
assert z >= 35 and k >= 35
######################################################
w = np.array([[5], [2], [1], [5]])
y = np.array([[0.5], [2], [-3]])
X = np.array([[1, 1, 1, 1], [0, 0, 0, 1], [3, 5, 3, 1]])

z , k, f = 0 , 0, 0
for i in range(100):
    g = stochastic_step(w, X, y)
    if g[0] == 108:
        z += 1
    elif g[0] == 0:
        k += 1
    else:
        f += 1
assert z >= 25 and k >= 25 and f >= 25

# SLIDE (1) Градиентный спуск

Теперь реализуем Линейную регрессию на градиентном спуске. Реализуйте пересчёт весов в цикле для алгоритма градиентного спуска. Начальные веса инициализированы нулями. 
$$\boldsymbol{w}^{(t+1)} = \boldsymbol{w}^{(t)} - \eta\nabla_{\boldsymbol{w}} L$$

Используйте параметры `max_iter`=$1000$, `eta`=$0.1$

Все итерации проходить не нужно. Остановитесь в момент, когда норма разницы между старыми и новыми весами станет меньше $1e-9$. 


### Sample 1
#### Input:
```python
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])

model = GDLinReg(max_iter=1000, eta=0.1).fit(X_train, y_train)
y_pred = model.predict(np.array([[3],[4]]))

```
#### Output:
```python
y_pred = np.array([3., 4.])
model.coef_ = np.array([1., 0.])
```

# TASK

In [44]:
class GDLinReg():
    def __init__(self, max_iter=1000, eta=0.1):
        self.max_iter = max_iter
        self.eta = eta
        self.coef_ = None #строка
        self.w_ = None #столбец
    
    def _gradient_descending(w, X, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
    
    def fit(self, X, y):
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        n, m = X.shape
        self.w_ = np.zeros((m, 1)) #столбец
        
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        self.coef_ = self.w_.reshape(-1)
        return self
        
    def predict(self, X):
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)   
        return (X @ self.w_).reshape(-1) #возвращаем всегда строку

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import make_regression
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = GDLinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[3],[4]])), np.array([3., 4.]), decimal=1)

assert_array_almost_equal(model.coef_, np.array([1., 0.]), decimal=1)
######################################################
X_reg, y_reg = make_regression(n_samples=200, n_features=10, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model2 = GDLinReg().fit(X_reg, y_reg)

coef_real = np.hstack([model.coef_, model.intercept_])
coef_my = model2.coef_

assert mean_absolute_error(coef_my, coef_real) < 5
######################################################

# SLIDE (1) Регуляризация

Реализуйте L2-регуляризацию (так же известную как Ridge).

$$L = \frac{1}{2}\lVert X\boldsymbol{w} - \boldsymbol{y}\rVert_2^2 + \frac{\lambda}{2}\lVert \boldsymbol{w} \rVert_2^2$$

Найдите новый $\nabla_{\boldsymbol{w}} L$ и верните его.

Обратите внимание, что свободный коэффициент весов (самый последний) **не нужно** регуляризовывать.

### Sample 1
#### Input:
```python
w = np.array([[1], [1]])
X = np.array([[1,1], [2,2], [3,3]])

y = np.array([[1],[0],[0]])
```
#### Output:
```python
np.array([[28.], 
          [27.]]) #возвращаем столбец!
```

# TASK

In [3]:
def gradient_step_l2(w: np.array, X: np.array, y: np.array, lamb: float) -> np.array:
    # Коэффициент регуляризации lamb
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [63]:
w = np.array([[1], [1]])
X = np.array([[1,1], [2,2], [3,3]])

y = np.array([[1],[0],[0]])
assert_array_almost_equal(gradient_step_l2(w, X, y, 1), np.array([[28.], [27.]]))

def gradient_descending(w, X, y, l):
    # посчитайте градиент функции ошибок по весам. Переменная l=0, не используйте её.
    g = X.T @ (X @ w - y)
    return g

w = np.array([[-1.90059439], [-0.39772511], [-0.41752917], [ 1.52698708],
              [ 1.93384661], [ 0.55038402], [-0.43914043], [-0.53450491],
              [-0.20436079], [ 1.47930748], [-0.43431298], [-0.67715663],
              [ 0.6140469 ], [ 0.33724021], [-0.47389417], [-0.27931867],
              [-0.19815547], [-1.10817221], [-0.2783202 ], [-0.26673129]])

y = np.array([[-1.35285676], [-1.90639143], [-1.1872863 ], [-0.47531246],
              [-0.5658066 ], [-2.01777789], [-0.55111576], [-1.17508504],
              [-0.31591632], [-1.10876769]])

X = np.array([[-1.79418438, -0.28247459,  0.63485396,  0.18973693, -1.13052678,
                -0.70693165,  0.02485169, -1.32269407, -1.19972889, -0.90835517,
                1.70735351, -0.31278511,  0.26195237,  1.15186451, -1.06028399,
                0.35805288,  0.1097917 , -1.64881113, -0.26764338,  0.52419335],
              [-0.39820007, -1.07706627,  0.03495314, -0.36069043, -0.38655026,
                0.99804715,  0.62738777,  1.53889357,  0.47893302, -1.17044503,
                -1.14344307,  0.99980883, -0.89283077, -0.30342722, -1.64897791,
                -1.13511464, -0.82071543,  1.8879236 , -1.5129215 ,  0.12872281],
              [-0.26032624, -0.59442306, -0.20607555,  1.0785192 , -0.72817743,
                1.60260617, -2.34932775,  0.55693193,  0.25573628,  1.53147877,
                1.53834672,  0.1878024 ,  1.18042765,  0.54514643,  0.01803304,
                1.51429761, -0.17469223,  0.81986536, -0.46578708,  0.42436921],
              [ 0.10641183, -0.18608143,  2.69988291,  0.30114113, -0.56763953,
                -0.89837188,  1.19577105, -0.86920194, -2.68921283, -0.58902866,
                -0.14408206, -1.84817003, -0.9124478 ,  0.55288917,  1.14731704,
                -0.4114787 , -0.53848335,  0.63225441,  0.61095749, -0.16311904],
              [ 0.42409436,  1.00368214, -1.47558928,  0.01692064,  0.95318017,
                -0.04567367, -1.62167744, -0.20345068, -0.1389324 , -0.4342349 ,
                0.41877207, -1.37897995, -0.31973986, -1.6112183 , -0.18485078,
                -1.16219592, -1.08054026, -0.49210254, -1.52373135,  2.34733614],
              [ 0.19339171, -1.25994886, -1.3263278 , -1.56807527, -1.16484596,
                1.51848938, -0.40378262,  0.28889234,  0.10067917,  1.89112864,
                -1.87355328,  0.19510271,  1.13826496,  0.41762094,  0.61912712,
                -1.06733394, -0.22801313, -1.49211563, -0.97819834, -0.08353847],
              [ 0.5225887 ,  0.03288573, -0.1318777 ,  0.75014314, -1.36193554,
                -0.71094454,  2.09153476,  1.50715026,  0.03406876, -0.71503137,
                1.22186058,  0.9721076 , -0.33217577,  0.60094908, -0.19959933,
                -0.79264798, -0.80242992, -1.46592485,  0.00955109, -0.31271658],
              [ 1.07854603,  1.63387859, -0.68689403, -1.49026312, -0.05352863,
                0.74923652, -0.61221446,  0.25499529, -0.87862293,  2.30763454,
                -0.45654804, -0.49537167, -1.14185717,  0.09358794,  1.71154247,
                0.51549091, -0.26392429,  1.16472574, -0.53397584, -0.79062073],
              [-0.56480156, -1.83123029,  0.99887682, -0.60648359,  0.37281722,
                0.19015213,  0.08486153,  1.54633375, -0.22768608,  1.4061507 ,
                -0.07928461, -1.03523357,  0.44072784, -0.98970029, -0.10722007,
                1.54788238, -0.89526339, -1.14606415, -0.65972113,  0.96906237],
              [ 1.36608419,  0.09914266, -0.02822534,  0.97303833,  2.14361812,
                0.18005144,  0.94756695,  1.97038803, -1.00026315,  0.47603231,
                1.14925524, -1.16030033,  1.42723652,  0.00458275,  0.62078139,
                -2.47565446, -0.16698078,  0.25653049,  0.8664139 , -0.37928587]])

######################################################
nw = np.copy(w)
nw[-1] = 0
l = 1
assert np.allclose(gradient_descending(w, X, y, 0) + l * nw, gradient_step_l2(w, X, y, l))
l = 123
assert np.allclose(gradient_descending(w, X, y, 0) + l * nw, gradient_step_l2(w, X, y, l))
######################################################

# SLIDE (1) Логистический градиент

Теперь переходим к задаче классификации и Логистической регрессии. Мы уже знаем, что в нашем случае нужно минимизировать метрику $LogLoss$:
$$\ln{L} = -\sum_{k=1}^{n}y_i\ln{(\sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))} + (1 - y_k)\ln{(1 - \sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))}$$
где $\sigma(z) = \frac{1}{1 + e^{-z}}$, $y \in \{0,1\}$

Ваша задача взять **производную** этой функции и вернуть вектор градиентов $\nabla_{\boldsymbol{w}}\ln{L}$. Формулу необходимо вывести в векторном виде, чтобы считалось быстрее.

Обратите внимание: 

* $w$, $y$ - столбцы, а не строки. Ответ тоже столбец.
* В функции используется **натуральный** логарифм.
* Нужно посчитать полный градиент, а не стохастический.

### Sample 1
#### Input:
```python
w = np.array([0, 0])
X = np.array([[1,1], [2,2], [3,3]])
y = np.array([[1],[0],[0]])
```
#### Output:
```python
array([[2], 
       [2]]) #возвращаем столбец!
```

# TASK

In [33]:
import numpy as np

def log_gradient_step(w: np.array, X: np.array, y: np.array)-> np.array:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    #нужно вернуть столбец
    pass

In [65]:
w = np.array([[0], [0]])
X = np.array([[1,1], [2,2], [3,3]])

y = np.array([[1],[0],[0]])
assert_array_almost_equal(log_gradient_step(w, X, y), np.array([[2], [2]]))
######################################################
w = np.array([[-1.90059439], [-0.39772511], [-0.41752917], [ 1.52698708],
              [ 1.93384661], [ 0.55038402], [-0.43914043], [-0.53450491],
              [-0.20436079], [ 1.47930748], [-0.43431298], [-0.67715663],
              [ 0.6140469 ], [ 0.33724021], [-0.47389417], [-0.27931867],
              [-0.19815547], [-1.10817221], [-0.2783202 ], [-0.26673129]])

y = np.array([[-1.35285676], [-1.90639143], [-1.1872863 ], [-0.47531246],
              [-0.5658066 ], [-2.01777789], [-0.55111576], [-1.17508504],
              [-0.31591632], [-1.10876769]])

X = np.array([[-1.79418438, -0.28247459,  0.63485396,  0.18973693, -1.13052678,
                -0.70693165,  0.02485169, -1.32269407, -1.19972889, -0.90835517,
                1.70735351, -0.31278511,  0.26195237,  1.15186451, -1.06028399,
                0.35805288,  0.1097917 , -1.64881113, -0.26764338,  0.52419335],
              [-0.39820007, -1.07706627,  0.03495314, -0.36069043, -0.38655026,
                0.99804715,  0.62738777,  1.53889357,  0.47893302, -1.17044503,
                -1.14344307,  0.99980883, -0.89283077, -0.30342722, -1.64897791,
                -1.13511464, -0.82071543,  1.8879236 , -1.5129215 ,  0.12872281],
              [-0.26032624, -0.59442306, -0.20607555,  1.0785192 , -0.72817743,
                1.60260617, -2.34932775,  0.55693193,  0.25573628,  1.53147877,
                1.53834672,  0.1878024 ,  1.18042765,  0.54514643,  0.01803304,
                1.51429761, -0.17469223,  0.81986536, -0.46578708,  0.42436921],
              [ 0.10641183, -0.18608143,  2.69988291,  0.30114113, -0.56763953,
                -0.89837188,  1.19577105, -0.86920194, -2.68921283, -0.58902866,
                -0.14408206, -1.84817003, -0.9124478 ,  0.55288917,  1.14731704,
                -0.4114787 , -0.53848335,  0.63225441,  0.61095749, -0.16311904],
              [ 0.42409436,  1.00368214, -1.47558928,  0.01692064,  0.95318017,
                -0.04567367, -1.62167744, -0.20345068, -0.1389324 , -0.4342349 ,
                0.41877207, -1.37897995, -0.31973986, -1.6112183 , -0.18485078,
                -1.16219592, -1.08054026, -0.49210254, -1.52373135,  2.34733614],
              [ 0.19339171, -1.25994886, -1.3263278 , -1.56807527, -1.16484596,
                1.51848938, -0.40378262,  0.28889234,  0.10067917,  1.89112864,
                -1.87355328,  0.19510271,  1.13826496,  0.41762094,  0.61912712,
                -1.06733394, -0.22801313, -1.49211563, -0.97819834, -0.08353847],
              [ 0.5225887 ,  0.03288573, -0.1318777 ,  0.75014314, -1.36193554,
                -0.71094454,  2.09153476,  1.50715026,  0.03406876, -0.71503137,
                1.22186058,  0.9721076 , -0.33217577,  0.60094908, -0.19959933,
                -0.79264798, -0.80242992, -1.46592485,  0.00955109, -0.31271658],
              [ 1.07854603,  1.63387859, -0.68689403, -1.49026312, -0.05352863,
                0.74923652, -0.61221446,  0.25499529, -0.87862293,  2.30763454,
                -0.45654804, -0.49537167, -1.14185717,  0.09358794,  1.71154247,
                0.51549091, -0.26392429,  1.16472574, -0.53397584, -0.79062073],
              [-0.56480156, -1.83123029,  0.99887682, -0.60648359,  0.37281722,
                0.19015213,  0.08486153,  1.54633375, -0.22768608,  1.4061507 ,
                -0.07928461, -1.03523357,  0.44072784, -0.98970029, -0.10722007,
                1.54788238, -0.89526339, -1.14606415, -0.65972113,  0.96906237],
              [ 1.36608419,  0.09914266, -0.02822534,  0.97303833,  2.14361812,
                0.18005144,  0.94756695,  1.97038803, -1.00026315,  0.47603231,
                1.14925524, -1.16030033,  1.42723652,  0.00458275,  0.62078139,
                -2.47565446, -0.16698078,  0.25653049,  0.8664139 , -0.37928587]])

g = np.array([[-0.43431449],
       [-6.4967537 ],
       [-3.33915541],
       [-2.60685525],
       [-3.13767717],
       [ 8.9115332 ],
       [-4.30338709],
       [ 8.52721835],
       [-6.04197056],
       [ 8.94208373],
       [ 2.41288223],
       [-4.65486414],
       [ 5.81341316],
       [ 1.54813258],
       [-0.23116471],
       [-6.1655314 ],
       [-6.56704074],
       [-3.53911584],
       [-9.10686695],
       [ 4.83364865]])

assert_array_almost_equal(log_gradient_step(w, X, y), g)
######################################################

# SLIDE (1) Логистическая регрессия

Теперь реализуем Логистическую регрессию на стохастическом градиентном спуске.
$$L = -\ln{Likelihood} = -\sum_{k=1}^{n}y_k\ln{(\sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))} + (1 - y_k)\ln{(1 - \sigma(\boldsymbol{x}_k^{T}\boldsymbol{w}))} + \frac{\lambda}{2}\lVert \boldsymbol{w} \rVert_2^2 \rightarrow \min$$
где $\sigma(z) = \frac{1}{1 + e^{-z}}$, $y \in \{0,1\}$, $\lVert \boldsymbol{w} \rVert_2^2 = \sum_{i=1}^{m} w_i^2$= - квадрат эвклидовой нормы

Собственно вероятность принадлежности определенному классу:
$$P(y_k=1) = \sigma(\boldsymbol{x}_k^T \boldsymbol{w})~~~P(y_k=0) = 1 - \sigma(\boldsymbol{x}_k^T \boldsymbol{w})$$

Реализуйте пересчёт весов в цикле для алгоритма градиентного спуска. Начальные веса сгенерированны рандомно. 

$$\boldsymbol{w}^{(t+1)} = \boldsymbol{w}^{(t)} - \eta\nabla_{\boldsymbol{w}} L_k$$

Будьте внимательны:

* На вход алгоритму приходит $\boldsymbol{y}$ - **строка**, как и в любой sklearn алгоритм.
* Не устанавливайте количество итераций больше 1000, так как алгоритм будет долго работать.
* Как и в линейной регрессии не забудьте добавить единичный столбец справа.

Можете использовать и обычный спуск, главное чтоб по времени зашло.

### Sample 1
#### Input:
```python
X_train = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_train = np.array([1, 1, 0, 0])

model = SGDLogReg().fit(X_train, y_train)
y_pred = model.predict(np.array([[0.5, 0.5], [ -0.5,  -0.5]]))
y_prob = model.predict_proba(np.array([[0.5, 0.5], [-0.5, -0.5]]))
```
#### Output:
```python
y_pred = np.array([1., 0.])
y_prob = np.array([[0.1, 0.9],  # это не точный ответ, но очень приблизительный, отличие на 0.1 - это нормально
                   [0.9, 0.1]]) # для каждого объекта возвращаем его вероятность нуля и единицы
```

# TASK

In [178]:
import numpy as np

class SGDLogReg():
    def __init__(self, max_iter=1000, eta=0.01, lamb = 1):
        self.eta = eta
        self.max_iter = max_iter
        self.lamb = lamb
        self.w_ = None
        self.coef_ = None
        
    def _stochastic_step(self, w: np.array, X: np.array, y: np.array) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        #модифицируйте функцию из предыдущей задачи
        pass
        
    def fit(self, X: np.array, y: np.array) -> np.array:
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        n, m = X.shape
        self.w_ = np.zeros(shape=(m, 1))
        
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        self.coef_ = self.w_.reshape(-1)
        return self
    
    def predict(self, X: np.array) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
    
    def predict_proba(self, X: np.array) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import mean_absolute_error
######################################################
X_clf = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_clf = np.array([1, 1, 0, 0])
model = SGDLogReg().fit(X_clf, y_clf)

assert model.lamb == 1

assert_equal(model.predict(np.array([[-0.5, -0.5]])), np.array([0.]))
assert_equal(model.predict(np.array([[ 0.5,  0.5]])), np.array([1.]))
######################################################
np.random.seed(1337)
n = 200
a = np.random.normal(loc=0, scale=1, size=(n, 2)) #первый класс
b = np.random.normal(loc=4, scale=2, size=(n, 2)) #второй класс
X = np.vstack([a, b]) #двумерный количественный признак
y = np.hstack([np.zeros(n), np.ones(n)]) #бинарный признак

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=1645)

model = SGDLogReg(lamb=0.01).fit(X_train, y_train)
model_real = LogisticRegression(C=0.01, solver='sag').fit(X_train, y_train)

assert model.lamb == 0.01

assert mean_absolute_error(model.predict(X_test), model_real.predict(X_test)) < 0.1
assert mean_absolute_error(model.predict_proba(X_test), model_real.predict_proba(X_test)) < 0.2
######################################################
np.random.seed(1228)
n = 1000
a = np.random.normal(loc=0, scale=1, size=(n, 2)) #первый класс
b = np.random.normal(loc=4, scale=1, size=(n, 2)) #второй класс
X = np.vstack([a, b]) #двумерный количественный признак
y = np.hstack([np.zeros(n), np.ones(n)]) #бинарный признак

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=1645)

model = SGDLogReg(lamb=0.5).fit(X_train, y_train)
model_real = SGDClassifier(loss='log',alpha=0.5).fit(X_train, y_train)


assert model.lamb == 0.5


assert mean_absolute_error(model.predict(X_test), model_real.predict(X_test)) < 0.2
assert mean_absolute_error(model.predict_proba(X_test), model_real.predict_proba(X_test)) < 0.25
######################################################

# SLIDE (1) Momentum

Этот метод позволяет направить sgd в нужной размерности и уменьшить осцилляцию. 

В общем случае он будет выглядеть следующим образом: 

$$ \boldsymbol{g}^{(t)} = \gamma \boldsymbol{g}^{(t - 1)} + \eta \nabla_{\boldsymbol{w}}L_{k}$$
$$ \boldsymbol{w}^{(t+1)} = \boldsymbol{w}^{(t)} - \boldsymbol{g}^{(t)}$$

где

 - $\eta$ — learning rate
 - $\boldsymbol{w}$ — вектор параметров
 - $\boldsymbol{g}$ — вектор градиентов 
 - $L$ — оптимизируемый функционал
 - $\gamma$ — momentum term (обычно выбирается 0.9)


# TASK

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SGDMomentum(BaseEstimator, ClassifierMixin):
    def __init__(self, features_size, gradient, lr=0.01, l=1, gamma=0.9, max_iter=1000):
        self.gradient = gradient #функция находящая градиент за вас
        self.lr = lr
        self.l = l
        self.gamma = gamma
        self.max_iter = max_iter
        self.w = np.random.normal(size=(features_size + 1, 1))

    def fit(self, X, y):
        v = np.zeros(self.w.shape)
        X = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
        for i in range(self.max_iter):
            index = np.random.randint(X.shape[0])
            cur_grad = self.gradient(self.w, X[index, :], np.array(y[index]), self.l)
            # пересчитайте веса в стохаистическом градиентном спуске
            '''
            .∧＿∧ 
            ( ･ω･｡)つ━☆・*。 
            ⊂  ノ    ・゜+. 
            しーＪ   °。+ *´¨) 
                    .· ´¸.·*´¨) 
                    (¸.·´ (¸.·'* ☆  <YOUR CODE>
            '''
            self.w = 
        return self

In [69]:
X = np.array([[ 2.67973871e-01, -9.43407158e-01,  4.59566449e-01,
         1.63136986e-01, -5.44506820e-01]])
y = np.array([-1])

r0 = SGDMomentum(5, lambda a, b, c, d: a, max_iter=10, l=1, lr=1)
r0.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r0.fit(X, y)
r1 = SGDMomentum(5, lambda a, b, c, d: a, max_iter=10, l=1, lr=0.1)
r1.w = np.array([[0.1], [0.2], [0.3], [0.2], [0.1], [-0.1]])
r1.fit(X, y)
######################################################
assert np.allclose(r0.w.reshape(6), np.array([ 0.01753165,  0.0350633,   0.05259494,  0.0350633,   0.01753165, -0.01753165]))
assert np.allclose(r1.w.reshape(6), np.array([-0.05887894, -0.11775788, -0.17663682, -0.11775788, -0.05887894,  0.05887894]))
######################################################